## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откроем файл с данными и изучим общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv') 
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Датасет состоит из 12 столбцов, в которых по 21 525 строк. 2 столбца содержат нулевые данные, в них по 19 351 не нулевых строк, соответственно нулевых 21 525 - 19 351 = 2174 строк. В следующем шаге будем заполнять пропуски. Считаю что просто отбросить их нельзя, так как это 10% ценной информации (например количество детей и наличие задолженности). total_income можно заполнить средними значениями в группировке по income_type. days_employed в зависимости от возраста dob_years.

In [2]:
data[['days_employed','dob_years']].describe()

,days_employed,dob_years
count,19351.000000,21525.000000
mean,63046.497661,43.293380
std,140827.311974,12.574584
min,-18388.949901,0.000000
25%,-2747.423625,33.000000
50%,-1203.369529,42.000000
75%,-291.095954,53.000000
max,401755.400475,75.000000


In [3]:
data.groupby('income_type')['days_employed'].mean()

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -3399.896902
компаньон           -2111.524398
пенсионер          365003.491245
предприниматель      -520.848083
сотрудник           -2326.499216
студент              -578.751554
Name: days_employed, dtype: float64

Посмотрели статистику по столбцам days_employed и dob_years
Предполгаю ошибку в выгрузке. Необходимо отрицательные числа взять по модулю.
У пенсионеров и безработных слишком большие значения, скорее всего это данные в часах, т.к. 365 503 / 24 / 365 =41 год.

В рабочем проекте запросили бы у клиента подтверждение догадок или попросили бы перевыгрузить данные корректно.

In [4]:
pd.options.mode.chained_assignment = None 
data['days_employed'][data['days_employed'] < 0] = abs(data['days_employed']) 
data['days_employed'][data['days_employed'] > 365*70] = data['days_employed'] / 24 
data['dob_years'][data['dob_years'] == 0] = data['dob_years'].mean()
data_drop = data.dropna(subset = ['days_employed', 'total_income']) 
data_drop['days_employed'] = data_drop['days_employed'].astype('int64')
data_drop[['days_employed','dob_years']].describe()

,days_employed,dob_years
count,19351.000000,19351.000000
mean,4641.141905,43.458927
std,5355.965045,12.222723
min,24.000000,19.000000
25%,926.500000,33.000000
50%,2194.000000,43.000000
75%,5537.000000,53.000000
max,18388.000000,75.000000


Отрицательное кол-во дней делаем положительным. Аномально большое количество дней разделили на 24, так как это скорее всего часы. Возраст равный 0 заполнили средним. Отбросили пустые строчки, чтобы посмотреть есть ли кореляция между возрастом и стажем.

Количество дней в days_employed стало нормальным, но необходимо еще заполнить отсутствующие значения

In [5]:
from scipy.stats.stats import pearsonr
print(pearsonr(data_drop['days_employed'], data_drop['dob_years'])) 

(0.6524052537172268, 0.0)


Рассчитали кареляцию между возрастом и стажем. Есть кореляция, p-value < 0.05, будем заполнять по линейной регрессии в зависимости от возраста

### Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
from scipy.stats import linregress
x = data_drop.dob_years
y = data_drop.days_employed
slope, intercept, r, p_value, std_err = linregress(x, y) 
print(slope, intercept, r, p_value, std_err)
data.days_employed = data.days_employed.fillna(intercept + slope * data.dob_years) 
data.days_employed[data.days_employed<0] = 0 
print(data.days_employed.describe())

count    21525.000000
mean      4659.870291
std       5189.736694
min          0.000000
25%        956.536601
50%       2360.703095
75%       6116.108075
max      18388.949901
Name: days_employed, dtype: float64


Рассчитали линейную решрессию. Заполнили пустые значения по линейной регрессии. На слишком маленький возраст заполняется отрицательный стаж, поставили 0.

Пропуски в days_employed заполнены, переходим на total_income. Записали уникальные типы дохода в переменную types. Цикл пробегает по всем типам дохода и записывает по нему медиану.

In [7]:
types = data.income_type.unique() 
print('Заполням такими средними значениями:')
for item in types:
    median = data.loc[data['income_type']==item]['total_income'].median() 
    print(item, median)
    data.loc[data['income_type']==item] = data.loc[data['income_type']==item].fillna(median)      
print('____________________')
print(data.info())

Заполням такими средними значениями:
сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
____________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB
None


Проверка показывает, что пустых значений больше нет.

### В датасете были найдены пропущенные значения и аномалии. В столбце dob_years были обнаружены 0. Они были заменены на среднее арифметическое. В столбце days_employed отрицательные значения были заменены на положительные, аномально большие разделены на 24 (часы), пропуски были заполнены методом линейной регрессии отталкиваясь от возраста. В столбце total_income пропуски были заполнены средними по аналогичным income_type.

### Замена типа данных

In [8]:
data.days_employed = data.days_employed.astype('int64')
data.dob_years = data.dob_years.astype('int64')
data.total_income = data.total_income.astype('int64')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Все вещественные типы были заменены на целочисленные методом astype потому что с целочисленными удобнее работать. Возраст и стаж у нас не дробный, копейки в доходе можно отбросить

### Обработка дубликатов

In [9]:
print('До преобразования:')
print(data.groupby('education')['education_id'].mean())
data.education = data.education.str.lower()
data.purpose = data.purpose.str.lower()
print('После преобразования:')
print(data.groupby('education')['education_id'].mean())

До преобразования:
education
ВЫСШЕЕ                 0
Высшее                 0
НАЧАЛЬНОЕ              3
НЕОКОНЧЕННОЕ ВЫСШЕЕ    2
Начальное              3
Неоконченное высшее    2
СРЕДНЕЕ                1
Среднее                1
УЧЕНАЯ СТЕПЕНЬ         4
Ученая степень         4
высшее                 0
начальное              3
неоконченное высшее    2
среднее                1
ученая степень         4
Name: education_id, dtype: int64
После преобразования:
education
высшее                 0
начальное              3
неоконченное высшее    2
среднее                1
ученая степень         4
Name: education_id, dtype: int64


Поиск соответствий family_status и family_status_id:

In [10]:
print(data.groupby('family_status')['family_status_id'].mean())

family_status
Не женат / не замужем    4
в разводе                3
вдовец / вдова           2
гражданский брак         1
женат / замужем          0
Name: family_status_id, dtype: int64


In [11]:
print(data.groupby('gender')['gender'].count())
i = (data[data['gender'] == 'XNA']).index
data = data.drop(i)
print(data.groupby('gender')['gender'].count())

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
gender
F    14236
M     7288
Name: gender, dtype: int64


Удалено лицо без пола. Это всего 1 запись и не повлияет на результаты исследования. Далее осуществили поиск и удаление дубликатов:

In [12]:
dup=data[data.duplicated(keep=False)] 
data = data.drop_duplicates().reset_index(drop=True) 
print(data.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
children            21453 non-null int64
days_employed       21453 non-null int64
dob_years           21453 non-null int64
education           21453 non-null object
education_id        21453 non-null int64
family_status       21453 non-null object
family_status_id    21453 non-null int64
gender              21453 non-null object
income_type         21453 non-null object
debt                21453 non-null int64
total_income        21453 non-null int64
purpose             21453 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Прошли по всем столбцам типа object методом groupbuy. Кроме цели кредита (будет рассматриваться далее) и кроме income_type (уже рассмотрено в процессе заполнения пропусков в доходе). Выявлены различия в регистрах в написании образования. Все было приведено в нижний регистр. Также был выявлен неопределенный пол. Так как такая строка всего 1, то она была удалена. После того, как все привели к одном регистру, осуществили поиск дубликатов методом duplicated и удалили их методом drop_duplicates. Разный регистр скорее всего из-за человеческого фактора, так как все люди вводят по-разному. 

### Лемматизация

In [13]:
from pymystem3 import Mystem
m = Mystem()
purposes = data['purpose'].unique()
lemmas = []
for item in purposes:
    lemmas += m.lemmatize(item)
lemmas = ' '.join(lemmas)
lemmas

'покупка   жилье \n приобретение   автомобиль \n дополнительный   образование \n сыграть   свадьба \n операция   с   жилье \n образование \n на   проведение   свадьба \n покупка   жилье   для   семья \n покупка   недвижимость \n покупка   коммерческий   недвижимость \n покупка   жилой   недвижимость \n строительство   собственный   недвижимость \n недвижимость \n строительство   недвижимость \n на   покупка   подержать   автомобиль \n на   покупка   свой   автомобиль \n операция   с   коммерческий   недвижимость \n строительство   жилой   недвижимость \n жилье \n операция   со   свой   недвижимость \n автомобиль \n заниматься   образование \n сделка   с   подержанный   автомобиль \n получение   образование \n автомобиль \n свадьба \n получение   дополнительный   образование \n покупка   свой   жилье \n операция   с   недвижимость \n получение   высокий   образование \n свой   автомобиль \n сделка   с   автомобиль \n профильный   образование \n высокий   образование \n покупка   жилье  

<font color='blue'>Получил основные цели: жилье, автомобиль, образование, свадьба, недвижимость, ремонт
</font>


In [14]:
def purpose_credit(purpose):
    purp = m.lemmatize(purpose)
    if 'свадьба' in purp:
        return 'свадьба'
    if 'ремонт' in purp:
        return 'ремонт'
    if 'жилье' in purp:
        return 'жилье'
    if 'автомобиль' in purp:
        return 'автомобиль'
    if 'образование' in purp:
        return 'образование'
    if 'недвижимость' in purp:
        return 'недвижимость'
data['purpose'] = data['purpose'].apply(purpose_credit)                        
data.groupby('purpose')['purpose'].count()
        

purpose
автомобиль      4306
жилье           3853
недвижимость    6350
образование     4013
ремонт           607
свадьба         2324
Name: purpose, dtype: int64

### Разбили цели на основные леммы и вывели статистику. Чаще всего берут кредит на жилье+недвижимость, затем автомобиль, образование, свадьбу, ремонт.

### Категоризация данных

Создадим словари family_status_dict и education_dict:

In [15]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict.sort_values('family_status_id', ascending=True))
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict.sort_values('education_id', ascending=True))

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем
   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень


затем можно в основном датасете удалить столбцы education и family_status для оптимизации таблицы и экономии места:

In [16]:
data.drop(['education', 'family_status'], axis = 'columns', inplace = True)
print('Датасет после удаления столбцов:')
print(data.head(5))

Датасет после удаления столбцов:
   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1           8437         42             0                 0      F   
1         1           4024         36             1                 0      F   
2         0           5623         33             1                 0      M   
3         3           4124         32             1                 0      M   
4         0          14177         53             1                 1      F   

  income_type  debt  total_income      purpose  
0   сотрудник     0        253875        жилье  
1   сотрудник     0        112080   автомобиль  
2   сотрудник     0        145885        жилье  
3   сотрудник     0        267628  образование  
4   пенсионер     0        158616      свадьба  


Разобьем на категории по возрастам, чтобы можно было смотреть статистку в каком возрастном диапазоне чаще имеют проблемы
с кредитами. Для этого создадим функцию для определения возрастной категории:

In [17]:
def age_cat(age):
    if age < 25:
        return 'менее 25'
    if age < 35:
        return 'от 25 до 34'
    if age < 45:
        return 'от 35 до 44'
    if age < 55:
        return 'от 45 до 54'
    if age < 65:
        return 'от 55 до 64'
    return '65 и старше'
data['age_category'] = data ['dob_years'].apply(age_cat)
data_age = data.groupby('age_category').agg({'debt':['count', 'sum']})
data_age['conversion'] = data_age['debt']['sum'] / data_age['debt']['count'] 
data_age.columns = ['Общее кол-во', 'Кол-во должников', '% должников']
data_age.style.format({'% должников': '{:.2%}'}) 

,Общее кол-во,Кол-во должников,% должников
age_category,,,
65 и старше,895,49,5.47%
менее 25,874,88,10.07%
от 25 до 34,5092,563,11.06%
от 35 до 44,5841,482,8.25%
от 45 до 54,4867,345,7.09%
от 55 до 64,3884,214,5.51%


Разобьем на категории по уровню дохода:

In [18]:
def income_cat(income):
    if income < 100000:
        return 'менее 100 тысяч рублей'
    if income < 150000:
        return 'от 100 до 149,99 тысяч рублей'
    if income < 200000:
        return 'от 150 до 199,99 тысяч рублей'
    if income < 250000:
        return 'от 200 до 249,99 тысяч рублей'
    return 'от 250 тысяч рублей'
data['income_category'] = data['total_income'].apply(income_cat)
print(data.head(10))

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1           8437         42             0                 0      F   
1         1           4024         36             1                 0      F   
2         0           5623         33             1                 0      M   
3         3           4124         32             1                 0      M   
4         0          14177         53             1                 1      F   
5         0            926         27             0                 1      M   
6         0           2879         43             0                 0      F   
7         0            152         50             1                 0      M   
8         2           6929         35             0                 1      F   
9         0           2188         41             1                 0      M   

  income_type  debt  total_income      purpose age_category  \
0   сотрудник     0        253875        жилье  от 35 до

### Для оптимизации таблицы и экономии места были созданы словари education_dict и family_status_dict. Столбцы education и family_status были удалены. Таблица стала компактней, наглядней. Ранее 12 столбцов помещались только в 3 строки, теперь в 2. 
### Также был создан новый столбец age_category и все данные были категоризированы по возрастным категориям. Из полученных данных можно сделать вывод, что чаще всего имеют плохие долги люди в возрасте от 25 до 34 лет (11,056% плохих долгов).
### Кроме того, был создан столбец income_category и данные были разбиты по уровню дохода. Анализ влияния уровня дохода на возврат кредита будет сделан в шаге 3.

### Шаг 3. Анализ влияния факторов на возврат кредита в срок.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
child_group = data.groupby('children').agg({'debt':['count', 'sum']})
child_group['conversion'] = child_group['debt']['sum'] / child_group['debt']['count']
child_group.drop([-1, 20], inplace=True) 
print(child_group)
x = [0, 1, 2, 3, 4, 5]
print('коэффициент кореляции между количеством детей и невозвратом кредита:'
      , pearsonr(x, child_group['conversion']))

           debt       conversion
          count   sum           
children                        
0         14090  1063   0.075444
1          4808   444   0.092346
2          2052   194   0.094542
3           330    27   0.081818
4            41     4   0.097561
5             9     0   0.000000
коэффициент кореляции между количеством детей и невозвратом кредита: (-0.5404537074698131, 0.26825005722815376)


Удалил аномальное количество детей -1 и 20

In [20]:
data.pivot_table('debt', index = 'children')


,debt
children,
-1,0.021277
0,0.075444
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


### Визуально видно, что % невозврата скачет разнонраправленно от изменения кол-ва детей. Коээфициент кореляции далек от -1 и p-value > 0,05, а значит зависимость между количеством детей и % невозврата не линейная. Лучше отдают кредиты те, у кого нет детей или кол-во детей равно 3. А вот у кого 1,2 и 4 ребенка процент невозврата повыше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
family_group = data.groupby('family_status_id').agg({'debt':['count', 'sum']})
family_group['conversion'] = family_group['debt']['sum'] / family_group['debt']['count']
print(family_status_dict)
family_group

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем


debt      conversion
                  count  sum           
family_status_id                       
0                 12339  931   0.075452
1                  4150  388   0.093494
2                   959   63   0.065693
3                  1195   85   0.071130
4                  2810  274   0.097509

In [22]:
data.pivot_table('debt', index = 'family_status_id')

,debt
family_status_id,
0,0.075452
1,0.093494
2,0.065693
3,0.071130
4,0.097509


### Из полученных данных делаю вывод, что наибольший % невозвратов среди "не женат/не замужем" и "гражданский брак". У тех, кто когда либо официально оформлял отношения, % невозврата заметно ниже.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
data_income = data.groupby('income_category').agg({'debt':['count', 'sum']})
data_income['conversion'] = data_income['debt']['sum'] / data_income['debt']['count'] 
print(data_income)

                               debt      conversion
                              count  sum           
income_category                                    
менее 100 тысяч рублей         4463  354   0.079319
от 100 до 149,99 тысяч рублей  7160  624   0.087151
от 150 до 199,99 тысяч рублей  4764  405   0.085013
от 200 до 249,99 тысяч рублей  2253  164   0.072792
от 250 тысяч рублей            2813  194   0.068966


In [24]:
data.pivot_table('debt', index = 'income_category')

,debt
income_category,
менее 100 тысяч рублей,0.079319
"от 100 до 149,99 тысяч рублей",0.087151
"от 150 до 199,99 тысяч рублей",0.085013
"от 200 до 249,99 тысяч рублей",0.072792
от 250 тысяч рублей,0.068966


### Наихудший процент возврата кредита среди людей, получающих от 100 до 200 тысяч рублей. Среди людей получающих больше 200 и меньше 100 процент невозврата заметно ниже.

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
data_purpose = data.groupby('purpose').agg({'debt':['count', 'sum']})
data_purpose['conversion'] = data_purpose['debt']['sum'] / data_purpose['debt']['count']
print(data_purpose)

              debt      conversion
             count  sum           
purpose                           
автомобиль    4306  403   0.093590
жилье         3853  273   0.070854
недвижимость  6350  474   0.074646
образование   4013  370   0.092200
ремонт         607   35   0.057661
свадьба       2324  186   0.080034


In [26]:
data.pivot_table('debt', index = 'purpose')

,debt
purpose,
автомобиль,0.093590
жилье,0.070854
недвижимость,0.074646
образование,0.092200
ремонт,0.057661
свадьба,0.080034


### Самый высокий % невозвратов по кредитам на автомобиль и образование. Затем идет свадьба. И самые лучшие цели это ремонт, жилье, недвижимость.

### Шаг 4. Общий вывод

Была получена выгрузка от клиента. После анализа выяснилось, что имеются аномальные данные в стаже (отрицательные были заменены на положительные, аномально большие поделены на 24). Также имелись пропуски данных в стаже и общем доходе. Пропуски в стаже были заполнены по линейной регрессии в зависимости от возраста. Пропуски в доходе средним по типу дохода.
Были удалены дубликаты строк. Выделены леммы в целях погашения кредита. Данные были категоризированы по сумме дохода и по возрасту.Выявлены зависимости между наличием детей, семейного положения, уровнем дохода и цели на возврат кредита в срок. Наилучший заемщик это вдовец/вдова, не имеет детей, с зарплатой более 250 тысяч рублей который берет кредит на ремонт. Наихудший заемщик не женат/не замужем, имеет 4 детей, имеет доход от 100 до 200 тысяч рублей и берет кредит на автомобиль.